In [5]:
from sage.all import *
import numpy as np
import random

def kernPause(a1,a2):
    return  1*(a1==a2)

def kernPitch(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)**2/(a*b)

def kernVolume(v1,v2):
    #return kernJacc(v1,v2)
    return min(v1,v2) #/max(v1,v2)

def kernDuration(d1,d2):
    return min(d1,d2)

def create_random_graph(n):
    G = Graph(loops=False)
    G.add_vertex(1-1)
    for k in range(2,n+1):
        vert = [v for v in G.vertices()]
        G.add_vertex(k-1)
        for v in vert:
            prob = 1.0/v*k/sigma(k)
            p = randint(1,100)/100.0
            #print(prob,p)
            if p <= prob and k%(v+1)==0:
                G.add_edge(v,k-1)
    return G

def create_divisor_graph(n):
    G = DiGraph(loops=True)
    divs = divisors(n)
    for d in divs:
        G.add_vertex(divs.index(d))
    for u in divs:
        for v in divs:
            if is_prime(u%v):
                G.add_edge(divs.index(u),divs.index(v))
    return G

def kernAdd(t1,t2,alphaPitch=0.25):
    pitch1,volume1,isPause1 = t1
    pitch2,volume2,isPause2 = t2
    #return 1.0/3*(1-alphaPitch)*kernPause(isPause1,isPause2)+alphaPitch*kernPitch(pitch1,pitch2)+1.0/3*(1-alphaPitch)*kernDuration(duration1,duration2)+1.0/3*(1-alphaPitch)*kernVolume(volume1,volume2)
    apa = alphaPitch["pause"]
    api = alphaPitch["pitch"]
    avo = alphaPitch["volume"]
    #return kernPause(isPause1,isPause2)*kernPitch(pitch1,pitch2)*kernVolume(volume1,volume2)

    if np.abs(apa+api+avo-1)<10**-5:
        return apa*kernPause(isPause1,isPause2)+api*kernPitch(pitch1,pitch2)+avo*kernVolume(volume1,volume2)
    else:
        return None

def kern0(zz0,alphaPitch={"pitch":1,"volume":2,"pause":3}):
    return lambda t1,t2: kernAdd(zz0[int(t1[0])],zz0[int(t2[0])],alphaPitch)

def kern(alphaPitch={"pitch":1,"volume":2,"pause":3}):
    return lambda t1,t2: kernAdd(t1,t2,alphaPitch)



def distKern1(x,y,alphaPitch={"pitch":1,"volume":2,"pause":3}):
    #print(alphaPitch)
    return np.sqrt(2-2*kern(alphaPitch)(x,y))

def distKern(kern):
    return lambda a,b : np.sqrt(kern(a,a)+kern(b,b)-2*kern(a,b))

import music21 as m21
from itertools import product

durlist = [[sum([(power(2,(n-i))) for i in range(d+1)]) for n in range(-8,3+1)] for d in range(2)]
durationslist = []
for dl in durlist:
    durationslist.extend([x for x in dl if x<=1.5])
print(durationslist)

def findNearestDuration(duration,durationslist):
    return sorted([(abs(duration-nv),nv) for nv in durationslist])[0][1]

def parse_file(xml):
    xml_data = m21.converter.parse(xml)
    score = []
    for part in xml_data.parts:
        parts = []
        for note in part.recurse().notesAndRests:
            if note.isRest:
                start = note.offset
                duration = float(note.quarterLength)/4.0
                vol = 32 #note.volume.velocity
                pitch = 60
                parts.append([pitch,findNearestDuration(duration,durationslist),vol,True])
            else:
                #print(note)
                start = note.offset
                duration = float(note.quarterLength)/4.0
                pitch = note.pitch.midi
                #print(pitch,duration,note.volume)
                vol = note.volume.velocity
                if vol is None:
                    vol = int(note.volume.realized * 127)
                parts.append([pitch,findNearestDuration(duration,durationslist),vol,False])    
        score.append(parts)        
    return score


def writePitches(fn,inds,tempo=82,instrument=[0,0],add21=True,start_at= [0,0],durationsInQuarterNotes=False):
    from MidiFile import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = len(inds)
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(ni):
        MyMIDI.addProgramChange(k,k,0,instrument[k])


    times = start_at
    for k in range(len(inds)):
        channel = k
        track = k
        for i in range(len(inds[k])):
            pitch,duration,volume,isPause = inds[k][i]
            #print(pitch,duration,volume,isPause)
            track = k
            channel = k
            if not durationsInQuarterNotes:
                duration = 4*duration#*maxDurations[k] #findNearestDuration(duration*12*4)            
            #print(k,pitch,times[k],duration,100)
            if not isPause: #rest
                #print(volumes[i])
                # because of median:
                pitch = int(floor(pitch))
                if add21:
                    pitch += 21
                #print(pitch,times[k],duration,volume,isPause)    
                MyMIDI.addNote(track, channel, int(pitch), float(times[k]) , float(duration), int(volume))
                times[k] += duration*1.0  
            else:
                times[k] += duration*1.0
       
    with open(fn, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")  


def run_length_row(row_of_01):
    from itertools import groupby
    ar = row_of_01
    return [(k, sum(1 for i in g)) for k,g in groupby(ar)]

def int_comp_row(row_of_01,by=8):
    ll = divide_row_by(row_of_01,by=by)
    ss = []
    for l in ll:
        rl = run_length_row(l)
        ss.append([v for k,v in rl])
    return list(ss)    

def divide_row_by(row,by):
    ll = []
    n = len(row)
    m = n//by
    #print(m)
    for k in range(m):
        ll.append(row[k*by:((k+1)*by)])
    return list(ll)    

def generateNotes(pitchlist,alphaPitch={"pitch":1,"volume":2,"pause":3},shuffle_notes=True):
    from itertools import product
    from music21 import pitch
    #pitchlist = [p for p in list(range(60-1*octave*12,60+24-1*octave*12))]
    #distmat = np.array(matrix([[np.sqrt(2*(1.0-kernPitch(x,y))) for x in pitchlist] for y in pitchlist]))
    #permutation,distance = tspWithDistanceMatrix(distmat,exact=False)
    #pitchlist = [pitchlist[permutation[k]] for k in range(len(pitchlist))]
    print([pitch.Pitch(midi=int(p)) for p in pitchlist])
    #durationlist = [n for n in durs]
    #if len(durs)>2:
    #    distmat = np.array(matrix([[np.sqrt(2*(1.0-kernDuration(x,y))) for x in durationlist] for y in durationlist]))
    #    permutation,distance = tspWithDistanceMatrix(distmat)
    #    durationlist = [durationlist[permutation[k]] for k in range(len(durationlist))]
    #print(durationlist)
    volumelist = vols = [(128//8)*(k+1) for k in range(8)] #[x*127 for x in [1.0/6.0,1.0/3.0,1.0/2.0,2.0/3.0 ]]
    print(volumelist)
    #distmat = np.array(matrix([[np.sqrt(2*(1.0-kernVolume(x,y))) for x in volumelist] for y in volumelist]))
    #permutation,distance = tspWithDistanceMatrix(distmat)
    #volumelist = [volumelist[permutation[k]] for k in range(len(volumelist))]
    print(volumelist)
    pauselist = [False,True]
    ll = list(product(pauselist,volumelist,pitchlist))
    if shuffle_notes:
        shuffle(ll)
    #distmat = np.array(matrix([[distKern(x,y,alphaPitch) for x in ll] for y in ll]))
    #np.random.seed(43)
    #permutation,distance = tspWithDistanceMatrix(distmat,exact=False)
    #ll = [ll[permutation[k]] for k in range(len(ll))]
    print(len(ll))
    #print(ll)
    pitches = [p[2] for p in ll]
    #durations = [d[0] for d in ll]
    volumes = [v[1] for v in ll]
    isPauses = [p[0] for p in ll]
    #print(pitches)
    return pitchlist,volumelist,pauselist

def getRational(k):
    alpha = 2**(1/12.0)
    x = RDF(alpha**k).n(50)
    return x.nearby_rational(max_error=0.01*x)

def get_knn_model(for_list,kern=kernPitch):
    #notes = np.array([[x*1.0 for x in n] for n in notes])
    from sklearn.neighbors import NearestNeighbors
    np.random.seed(0)
    #nbrs = NearestNeighbors( algorithm='ball_tree',metric=distKern(kern(zz0,alphaPitch=alphaPitch))).fit([[r] for r in range(len(zz0))])
    M = matrix([[float(kern(a,b)) for a in for_list] for b in for_list],ring=RDF)
    #print(M)
    Ch = np.array(M.cholesky())
    nbrs = NearestNeighbors().fit(Ch)
    return nbrs, Ch

def findBestMatches(nbrs,new_row,n_neighbors=3):
    distances,indices = nbrs.kneighbors([np.array(new_row)],n_neighbors=n_neighbors)
    dx = sorted(list(zip(distances[0],indices[0])))
    #print(dx)
    indi = [d[1] for d in dx]
    #print(indi)
    #print(distances)
    #distances,indices = nbrs.query([np.array(new_row)],k=n_neighbors)
    return indi


from itertools import groupby

def get_flattened_durations_from_graph(G,permutation_of_vertices,sorted_reverse,by=16,shuffled=True):
    zz = sorted(list(zip(G.degree_sequence(),G.vertices())),reverse=sorted_reverse)
    #print(sorted(zz))
    if shuffled:
        shuffle(zz)
    A = G.adjacency_matrix(vertices = permutation_of_vertices)
    ll = []
    for row in A:
        #print(row)
        ll.extend([xx/by for xx in x] for x in (int_comp_row(row,by=by)))
    ss = []
    for l in ll:
        ss.extend(l)
    return ss    

def get_bitstring_from_durations(durs,denom=None):
    # convert reciprocal into rational number:
    # take the denominator
    # take the gcd of all denominators
    qq = ([QQ(d) for d in durs])
    dd = [q.denominator() for q in qq]
    D = denom if not denom is None else lcm(dd)
    nn = [q.numerator()*D/q.denominator() for q  in qq]
    xx = []

    print(D,nn)
    y = 0
    for n in nn:
        xx.extend(n*[y])
        y = 1*(not y)
    return xx,D    
    
 
def get_durations_from_graph(G,sorted_reverse,by=16,shuffled=True):
    zz = sorted(list(zip(G.degree_sequence(),G.vertices())),reverse=sorted_reverse)
    #print(sorted(zz))
    if shuffled:
        shuffle(zz)
    A = G.adjacency_matrix(vertices = [z[1] for z in zz])
    ss = []
    for row in A:
        print(row)
        xx = ((int_comp_row(row,by=by)))
        ll = []
        for x in xx: 
            ll.extend([xs/by for xs in x])
        ss.append(ll)  
    return ss    

[1/256, 1/128, 1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 3/512, 3/256, 3/128, 3/64, 3/32, 3/16, 3/8, 3/4, 3/2]


In [78]:
# flattened durations
by = 8
N = by*2*3
#G = graphs.RandomBarabasiAlbert(N,N//3) #create_random_graph(N)
#G = graphs.RandomIntervalGraph(N)
#G = create_divisor_graph(N)
#G = graphs.RandomGNM(N, 400, dense=False, seed=None)
pitches,volumes,isPauses = list(generateNotes(pitchlist=range(15,128-15),shuffle_notes=False))
#print(pitches)
durations = [1/16,1/8,1/4,1/2,1]
#print(len(notes))
#shuffle(notes)
nbrsPitch, chPitch = get_knn_model(pitches,kernPitch)
nbrsVolume, chVolume = get_knn_model(volumes,kernVolume)
#nbrsDuration, chDuration = get_knn_model(durations,kernDuration)
nbrsPause, chPause = get_knn_model(isPauses,kernPause)


scores = parse_file("./For_You.mid")


pitches0 = [[n[0] for n in notes] for notes in scores]
vols0 = [[n[2] for n in notes] for notes in scores]
pauses0 = [[n[3] for n in notes] for notes in scores]
durs0 = [[n[1] for n in notes] for notes in scores]
#durs0 =[ 5*(
#           2*(16*[1/16]+4*[4/16]+[1/16,2/16,1/16,4/16]*2)
#          +3*( 8*[2/16]+16*[1/16]+[1/16,2/16,1/16,4/16]*3)
#          )+
#         6*(
#           3*(16*[1/16]+4*[4/16]+[1/16,2/16,1/16,4/16]*3)
#          +2*( 8*[2/16]+16*[1/16]+[1/16,2/16,1/16,4/16]*2)
#         )
#       ]

#8     CyclicPermutationGroup(8)                   Cyclic
#8     D1 = CyclicPermutationGroup(4)
#      D2 = CyclicPermutationGroup(2)
#      G = direct_product_permgroups([D1,D2])      Abelian, non-cyclic
#8     D1 = CyclicPermutationGroup(2)
#      D2 = CyclicPermutationGroup(2)
#      D3 = CyclicPermutationGroup(2)
#      G = direct_product_permgroups([D1,D2,D3])   Abelian, non-cyclic
#8     DihedralGroup(4)                            Non-abelian
#8     QuaternionGroup()                           Quaternions, also DiCyclicGroup(2)
#gr = AbelianGroup([2,2,2])

D1 = CyclicPermutationGroup(2)
D2 = CyclicPermutationGroup(2)
D3 = CyclicPermutationGroup(4)
#gr = direct_product_permgroups([D1,D2,D3]) 
gr = CyclicPermutationGroup(16) #16)8)
#gr = QuaternionGroup()

print(str(gr))

for g in gr:
    M = (g.matrix())
    print("---")
    print(M)

nVoices = gr.order()
durs = []
iinds = []
#or nv in range(nVoices):
for g in gr:
    M = g.matrix()
    for d1 in durs0:
        print(d1)
        #bs,D1 = get_bitstring_from_durations(d1,denom=32)
        bs,D1 = get_bitstring_from_durations(d1,denom=16)
        print(D1)
        bbs = []
        jq = len(bs)//gr.order()
        jr = len(bs)%gr.order()
        for j in range(jq):
            v = vector(bs[j*gr.order():(j+1)*gr.order()])
            mv = [x for x in (M*v)]
            bbs.extend(mv)
        bbs.extend(bs[jq*gr.order():])    
        d2 = [r[1]/D1 for r in run_length_row(bbs)]
        durs.append(d2)
        iinds.append([])




pitches1 = []
volumes1 = []
durations1 = []
pauses1 = []


for d in range(len(durs)):
    pitchBuffer = []
    lenBuffers = [30]
    #pitches1 = []
    #volumes1 = []
    #durations1 = []
    #pauses1 = []
    startPitch = chPitch[0]
    startVolume = chVolume[0]
    startPause = chPause[0]
    s = 0
    for k in range(len(durs[d])):
        duration1 = durs[d][k]
        s+=duration1
        vPitch  =  int(sum(Integer(k+d+1).digits(5)))+1
        vVolume =  int(sum(Integer(k+d+1).digits(2)))+1
        #v = int(duration1*by)
        #v = sigma(euler_phi(k+d+1))
        print(v)
        nextPitch = findBestMatches(nbrsPitch,startPitch,n_neighbors=min(len(pitches),vPitch))
        nextVolume = findBestMatches(nbrsVolume,startVolume,n_neighbors=min(len(volumes),vVolume))
        nextPause = findBestMatches(nbrsPause,startPause,n_neighbors=min(len(isPauses),2))
        pitches1.extend(nextPitch)
        volumes1.extend(nextVolume)
        pa0 = nextPause[0]
        #isPause = isPauses[pa0] or ( d > s**2*16.0/35**2  and s <= 35 ) or (s>35 and d>-8/35*s+24)
        #isPause = isPauses[pa0] or (d>-2.889908+4.251318*s-0.3608946*s**2+0.01179963*s**3-0.0001617849*s**4+7.823284*10**-7*s**5)
        #t = abs(s-77)
        t = s
        isPause = isPauses[pa0] or ( d > abs(cos(abs(s)*2*pi.n()/36)*abs(s/77)**(1/2.0)*16)+1)
        pauses1.append(isPause)
        pauses1.append(nextPause[1])
        p0 = pitches1.pop(0)
        v0 = volumes1.pop(0)
        #pa0 = pauses1.pop(0)
        pitch = pitches[p0]
        volume = volumes[v0]
        
        withBuffer = False
        if len(pitchBuffer)<=lenBuffers[0] and withBuffer:
            pitchBuffer.append(pitch)
        if len(pitchBuffer)>lenBuffers[0] and withBuffer:
            #lenBuffers.append(lenBuffers.pop(0))
            p = pitchBuffer.pop(0)
            m = int(floor(median(pitchBuffer)))
            octave = p//12
            pitchMod12 = p%12
            print(pitchMod12,m)
            pit = 12*octave + (7*pitchMod12+m)%12 #m-abs(pitch-m) if m > pitch else m+abs(pitch-m)
            pitchBuffer.append(pitch)
            pitch = pit
            
        startPitch = chPitch[nextPitch[0]]
        startVolume = chVolume[nextVolume[0]]
        startPause = chPause[pa0]
        note1 = pitch,duration1*1.0,volume,isPause
        print(note1)
        iinds[d].append(note1)

    
print(iinds)   

midiInstrumentNumber = 51 # synth strings
instr = (len(durs))*[80] # 51 synth strings
#instr.extend((nVoices//2)*[0]) # piano
#instr.extend((nVoices//4)*[0]) # sine wave (81)


fn1 = "./midi/experiment-"+str(gr).replace(" ","-")+"graph_"+str(N)+"-"+str(by)+"_"+str(nVoices)+".mid"
#fn = "../midi/experiment_in_intensity_for_piano.mid"
fn = fn1

writePitches(fn=fn,inds=iinds,tempo=64,instrument=instr,add21=True,start_at= len(durs)*[0],durationsInQuarterNotes=False)
    

[<music21.pitch.Pitch E-0>, <music21.pitch.Pitch E0>, <music21.pitch.Pitch F0>, <music21.pitch.Pitch F#0>, <music21.pitch.Pitch G0>, <music21.pitch.Pitch G#0>, <music21.pitch.Pitch A0>, <music21.pitch.Pitch B-0>, <music21.pitch.Pitch B0>, <music21.pitch.Pitch C1>, <music21.pitch.Pitch C#1>, <music21.pitch.Pitch D1>, <music21.pitch.Pitch E-1>, <music21.pitch.Pitch E1>, <music21.pitch.Pitch F1>, <music21.pitch.Pitch F#1>, <music21.pitch.Pitch G1>, <music21.pitch.Pitch G#1>, <music21.pitch.Pitch A1>, <music21.pitch.Pitch B-1>, <music21.pitch.Pitch B1>, <music21.pitch.Pitch C2>, <music21.pitch.Pitch C#2>, <music21.pitch.Pitch D2>, <music21.pitch.Pitch E-2>, <music21.pitch.Pitch E2>, <music21.pitch.Pitch F2>, <music21.pitch.Pitch F#2>, <music21.pitch.Pitch G2>, <music21.pitch.Pitch G#2>, <music21.pitch.Pitch A2>, <music21.pitch.Pitch B-2>, <music21.pitch.Pitch B2>, <music21.pitch.Pitch C3>, <music21.pitch.Pitch C#3>, <music21.pitch.Pitch D3>, <music21.pitch.Pitch E-3>, <music21.pitch.Pitch 

[1, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/8, 1/4, 1/4, 1/8, 1/8, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/8, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/8, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1

(43, 0.250000000000000, 32, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 48, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 64, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 32, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 48, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 64, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 80, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 32, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 48, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0

(46, 0.250000000000000, 80, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 96, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 32, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 48, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 64, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.250000000000000, 80, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 96, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(51, 0.250000000000000, 112, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 32, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 

(39, 0.250000000000000, 64, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 80, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 32, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(51, 0.250000000000000, 48, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(53, 0.250000000000000, 64, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 80, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 16, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 32, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 48, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 64, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.250000000000000, 80, False)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0

...

(34, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.250000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(51, 0.250000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(53, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(31, 0.250000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(55, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 80, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39,

(27, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.125000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.250000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.125000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.125000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.125000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.125000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.0625000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.0625000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.125000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(4

(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 80, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 96, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 112, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.125000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 32, True)
(0,

(34, 0.250000000000000, 80, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27, 0.250000000000000, 80, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(34, 0.250000000000000, 96, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(39, 0.250000000000000, 16, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(43, 0.250000000000000, 32, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(22, 0.250000000000000, 48, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(46, 0.250000000000000, 64, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(15, 0.250000000000000, 80, True)
(0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)
(27,

written
